In [1]:
# Library import
import xml.etree.ElementTree as ET
import spacy
import string
import numpy as np
import json
import random
import pickle
# for auto-correct words in review text
from autocorrect import Speller
spell = Speller()

2022-04-27 01:38:36.209209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-27 01:38:36.209277: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aryan/.local/lib/python3.8/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 

In [2]:
xml_filepath = './dataset/ABSA16_Laptops_Train_SB1_v2.xml'
# create element tree object
tree = ET.parse(xml_filepath)

# get root element
root = tree.getroot()

In [3]:
# # loading data from XML format to usable format -> review_text, entity, attribute, polarity(0/1)

# --->                             Skipping conflicting sentences               <---
# data = []
# max_review_length = 0
# for it1 in root.findall('Review'):
#     review = []
#     for it2 in it1.findall('sentences'):
#         for it3 in it2.findall('sentence'):
#             for it4 in it3.findall('text'):
#                 review_text = spell(it4.text.lower())

#             category_dict = {}
#             for it4 in it3.findall('Opinions'):
#                 for it5 in it4.findall('Opinion'):
#                     cat = it5.attrib['category']
                    
#                     # 0 negtaive, 1 neutral, 2 positive
#                     if it5.attrib['polarity'] == 'negative':
#                         polarity = 0
#                     elif it5.attrib['polarity'] == 'neutral':
#                         polarity = 1
#                     else:
#                         polarity = 2

#                     if category_dict.get(cat) == None:
#                         category_dict[cat] = polarity

#                     elif category_dict[cat] != -1:
#                         if category_dict[cat] != polarity:
#                             category_dict[cat] = -1

#             for cat in category_dict:
#                 if category_dict[cat] != -1:
#                     lst = [review_text]
#                     splitted_category = cat.split('#')
#                     for tok in splitted_category:
#                         lst.append(spell(tok.lower()))

#                     lst.append(category_dict[cat])
#                     review.append(lst)


#     max_review_length = max(max_review_length,len(review))
#     if len(review) != 0:
#         data.append(review)

# print('Max review length is: ', max_review_length)


# ->          Taking majority opinion in conflicting case for sentence-opinion pair       <-
data = []
max_review_length = 0
for it1 in root.findall('Review'):
    review = []
    for it2 in it1.findall('sentences'):
        for it3 in it2.findall('sentence'):
            for it4 in it3.findall('text'):
                review_text = spell(it4.text.lower())

            category_dict = {}
            for it4 in it3.findall('Opinions'):
                for it5 in it4.findall('Opinion'):
                    cat = it5.attrib['category']
                    
                    if it5.attrib['polarity'] == 'negative':
                        polarity = -1
                    elif it5.attrib['polarity'] == 'neutral':
                        polarity = 0
                    else:
                        polarity = 1

                    if category_dict.get(cat) == None:
                        category_dict[cat] = 0
                    category_dict[cat] += polarity

            # 0 negtaive, 1 neutral, 2 positive
            for cat in category_dict:
                lst = [review_text]
                splitted_category = cat.split('#')
                for tok in splitted_category:
                    lst.append(spell(tok.lower()))

                if category_dict[cat] == 0:
                    polarity = 1  #neutral
                elif category_dict[cat] > 0:
                    polarity = 2  #positive
                else:
                    polarity = 0  #negative

                lst.append(polarity)
                review.append(lst)

    max_review_length = max(max_review_length,len(review))
    if len(review) != 0:
        data.append(review)

print('Max review length is: ', max_review_length)

Max review length is:  25


In [4]:
print(len(data))

450


In [5]:
# sm is small here, can use large here as lg here by downloading the large file
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")


# 200 dimensions vectors for the tokens. Giving 0's to padding token and generating randomly for oov tokens.
emmbedding_dimensions = 300
emmbed_dict = {}
emmbed_dict['pad'] = np.zeros(emmbedding_dimensions)

with open('../dataset/glove.6B/glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:],'float64')
        emmbed_dict[word]=vector

emmbed_dict['pad'] = np.zeros(emmbedding_dimensions)

vec = []
for _ in range(emmbedding_dimensions):
    vec.append(random.uniform(-1,1))

emmbed_dict['oov'] = np.array(vec)

In [6]:
# removed punctuations
train_data_vocab = set()  #also building training data vocab
train_data_vocab.add('pad')
train_data_vocab.add('oov')
max_sent_length = 0
for i in range(len(data)):
    for j in range(len(data[i])):
        sent = data[i][j][0]

        token_lst = []
        for it in nlp(sent):
            token = it.text
            str = ''
            for ch in token:
                if ch == "'" or ch not in string.punctuation:
                    if ch == "'":
                        str += ' '
                    str += ch
                else:
                    str += ' '
            lst = str.split()
            for token in lst:
                if token in emmbed_dict:
                    token_lst.append(token)
                    train_data_vocab.add(token)
                else:
                    token_lst.append('oov')

        max_sent_length = max(len(token_lst), max_sent_length)
        data[i][j][0] = token_lst

        if data[i][j][1] in emmbed_dict:
            train_data_vocab.add(data[i][j][1])
        else:
            data[i][j][1] = 'oov'

        if data[i][j][2] in emmbed_dict:
            train_data_vocab.add(data[i][j][2])
        else:
            data[i][j][2] = 'oov'


print('Maximum Sentence Length is: ',max_sent_length)

Maximum Sentence Length is:  75


In [7]:
# token to index for training data
token_to_index = {}
for cnt, token in enumerate(train_data_vocab):
    token_to_index[token] = cnt

In [8]:
non_padded_sent_length = []
for i in range(len(data)):
    non_padded_sent_length.append(len(data[i]))

In [9]:
# review level padded thing
padded_sent = []
for i in range(max_sent_length):
    padded_sent.append('pad')
# neutral in pad 
padded_review = [padded_sent,'pad','pad',1]

# sentence level and review level padding
padded_data = []
temp_data = data.copy()
for i in range(len(data)):
    for j in range(max_review_length):
        if j < len(data[i]):
            token_lst = data[i][j][0]
            sent = []
            for k in range(max_sent_length):
                if k < len(token_lst):
                    sent.append(token_lst[k])
                else:
                    sent.append('pad')
            temp_data[i][j][0] = sent
        
        else:
            temp_data[i].append(padded_review)

    # inserting a token to acknowledge that after this point, only padded sentences occur
    padded_data.append([temp_data[i],non_padded_sent_length[i]])

In [10]:
review_indices = []
for i in range(len(padded_data)):
    reviews = []
    for j in range(len(padded_data[i][0])):
        review = []
        
        lst = []
        for token in padded_data[i][0][j][0]:
            lst.append(token_to_index[token])

        # token indices
        review.append(np.array(lst))
        # entitiy
        review.append(token_to_index[data[i][j][1]])
        # attribute
        review.append(token_to_index[data[i][j][2]])
        # polarity
        review.append(data[i][j][3])

        reviews.append(review)

    review_indices.append([reviews,padded_data[i][1]])

In [11]:
embedding_matrix = []
for token in train_data_vocab:
    embedding_matrix.append(emmbed_dict[token])

embedding_matrix = np.array(embedding_matrix)

In [12]:
# saving embedding matrix in numpy format
np.save('embedding_matrix_laptops',embedding_matrix)

In [13]:
dump_this_dict = {}
for i in range(len(review_indices)):
    dump_this_dict[i] = review_indices[i]

In [20]:
with open('train_laptops.json', 'wb') as f:
    pickle.dump(dump_this_dict, f)

In [21]:
with open('train_laptops.json', 'rb') as f:
    data = pickle.load(f)

In [22]:
# dumping token to index, so that we can create test data
with open('token_to_index_laptops.json', 'wb') as f:
    pickle.dump(token_to_index, f)

In [23]:
print(embedding_matrix.shape)

(2761, 300)


In [24]:
total = 0
pos = 0
neg = 0
neu = 0

for i in range(len(data)):
    for j in range(data[i][1]):
        if data[i][0][j][3] == 0:
            neg += 1
        elif data[i][0][j][3] == 1:
            neu += 1
        else:
            pos += 1

        total += 1

print(total, pos, neg, neu)

2906 1634 1081 191
